# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [3]:
# dataDir = 'data/2D_Truss_v1.3/conmech/design_7_N_1000/'
# allGraphsUnfiltered = loadConmechGraphs(dataDir)

dataDir = 'data/tower1.0/conmech/design_5_N_1000/'
allGraphsUnfiltered = loadConmechGraphs(dataDir, loadDims=[0])

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.027434
std,0.005930
min,0.022934
25%,0.023780
50%,0.025250
75%,0.028404
max,0.065064


In [10]:
plotTruss(allGraphsUnfiltered[0], 
          showDeformed=True, 
          defScale=100, 
          width=80, 
          lineWidth=1, 
          showLoads=True, 
          showSupports=True, 
          loadTheta=0, 
          loadSize=100, 
          supportSize=100)

alt.LayerChart(...)

## 2. Filter and partition

In [11]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.025836
std,0.002804
min,0.022934
25%,0.023687
50%,0.024775
75%,0.027176
max,0.034477


## 3. Train a GCN

In [16]:
gcn = FeaStNet()
# history = gcn.trainModel(trainData, valData, 
#                          epochs=100,
#                          saveDir='results/gcn02Tower/')

history = gcn.trainModel(trainData, trainData, 
                         epochs=100,
                         saveDir='results/gcn02TowerTEST/')

plotHistory(history)

epoch: 0   trainLoss: 6.0968e-01   valLoss:9.7328e-01  time: 4.87e+00
epoch: 1   trainLoss: 1.9355e-01   valLoss:8.7355e-01  time: 4.87e+00
epoch: 2   trainLoss: 8.8696e-02   valLoss:7.3880e-01  time: 4.77e+00
epoch: 3   trainLoss: 7.7561e-02   valLoss:6.3268e-01  time: 4.75e+00
epoch: 4   trainLoss: 6.2540e-02   valLoss:5.0210e-01  time: 4.76e+00
epoch: 5   trainLoss: 4.7231e-02   valLoss:3.8773e-01  time: 4.79e+00
epoch: 6   trainLoss: 4.1332e-02   valLoss:3.1293e-01  time: 4.82e+00
epoch: 7   trainLoss: 3.7475e-02   valLoss:2.2063e-01  time: 4.77e+00
epoch: 8   trainLoss: 2.9534e-02   valLoss:1.0734e-01  time: 4.78e+00
epoch: 9   trainLoss: 2.4739e-02   valLoss:5.3678e-02  time: 4.76e+00
epoch: 10   trainLoss: 2.1115e-02   valLoss:6.3456e-02  time: 4.82e+00
epoch: 11   trainLoss: 2.0578e-02   valLoss:6.6067e-02  time: 4.88e+00
epoch: 12   trainLoss: 1.7945e-02   valLoss:4.6350e-02  time: 4.84e+00
epoch: 13   trainLoss: 1.9219e-02   valLoss:4.0955e-02  time: 4.78e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [13]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,1.539682e-07,0.000323,0.020947,0.966817,0.973520,0.395071,-2.131005
test,1.535013e-07,0.000321,0.020801,0.961549,0.970831,0.409203,-1.821233


## 5. Visualize some predictions

In [14]:
i = 4
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=20, prediction=pred)

alt.LayerChart(...)

In [15]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');

alt.HConcatChart(...)